In [1]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D,Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session =tf.compat.v1.InteractiveSession(config=config)

In [3]:
input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)

#build convnet to use in each siamese neural newtork
model = Sequential()
model.add(Conv2D(64,(10,10),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D())
model.add(Conv2D(128,(7,7),activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128,(4,4),activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(256,(4,4),activation='relu'))
model.add(Flatten())
model.add(Dense(4096,activation="sigmoid"))#voltar pra 4096?

#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = model(left_input)
encoded_r = model(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid')(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(0.00006)
siamese_net.compile(loss="binary_crossentropy",optimizer='Adam',metrics=['accuracy'])

In [4]:
siamese_net.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 4096)         38947648    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 4096)         0           sequential[0][0]      

In [5]:
%run DataSetPrepare.ipynb

In [6]:
createLabelsTargets = CreateLabelsTargets("train_HB.csv")
createLabelsTargets.load_csv_and_treat() 
(X_train, X_test, y_train, y_test) = createLabelsTargets.create_labels_and_targets()

In [7]:
#Images path is equal, since test images doesn't have individual labels
imagesPath= os.getcwd()+'\\modelHB_imgs\\train\\'
# Creating training data generator
dataTrainLoader = DataLoader(imagesPath,X_train, y_train)
# Creating validating data generator
dataValidLoader = DataLoader(imagesPath,X_test, y_test)

In [8]:
#Training loop
evaluate_every = 200 # interval for evaluating on one-shot tasks
loss_every=200 # interval for printing loss (iterations)
batch_size = 32
n_iter = 90000
N_way = 5 # how many classes for testing one-shot tasks>
n_val = 150 #how mahy one-shot tasks to validate on?
print("training")
for i in range(1, n_iter):
    (inputs,targets)=dataTrainLoader.getBatch(batch_size)
    (loss,accuracy)=siamese_net.train_on_batch(inputs,targets)
    if i % evaluate_every == 0:
        print("evaluating")
        val_acc = dataTrainLoader.test_oneshot(siamese_net,N_way,n_val,X_test,y_test,verbose=True)

    if i % loss_every == 0:
        print("iteration {}, training loss: {:.2f}, val_acc: {:.2f}".format(i,loss,val_acc))


training


ResourceExhaustedError:  OOM when allocating tensor with shape[9216,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/functional_1/sequential/dense/MatMul_1 (defined at <ipython-input-8-c35ca4e93911>:11) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1222]

Function call stack:
train_function
